In [ ]:
# Copyright 2025 Arm Limited and/or its affiliates.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

# TOSA delegate flow example

This guide walks through the complete process of running a module on Arm TOSA using ExecuTorch, with a focus on TOSA lowering exploration. 
This workflow is intended for validating and experimenting with model lowering to TOSA, and is aimed at contributors and developers, rather than production deployment.
It’s important to note that the compilation flow and passes applied can vary based on the target, so this flow does not necessarily produce TOSA flatbuffers and PTE files which are optimal (or even compatible) with any one target.
If something is not working for you, please raise a GitHub issue and tag Arm.

Before you begin:
1. (In a clean virtual environment with a compatible Python version) Install executorch using `./install_executorch.sh`
2. Install Arm TOSA dependencies using `examples/arm/setup.sh --disable-ethos-u-deps`

With all commands executed from the base `executorch` folder.



*Some scripts in this notebook produces long output logs: Configuring the 'Customizing Notebook Layout' settings to enable 'Output:scrolling' and setting 'Output:Text Line Limit' makes this more manageable*

## AOT Flow

The first step is creating the PyTorch module and exporting it. Exporting converts the python code in the module into a graph structure. The result is still runnable python code, which can be displayed by printing the `graph_module` of the exported program.  

In [ ]:
import torch

print(torch.__version__)

class Add(torch.nn.Module):
    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        return x + y

example_inputs = (torch.ones(1,1,1,1),torch.ones(1,1,1,1))

model = Add()
model = model.eval()
exported_program = torch.export.export(model, example_inputs)
graph_module = exported_program.module()

_ = graph_module.print_readable()

## TOSA backend supports both INT and FP targets.

To lower the graph_module for FP targets using the TOSA backend, we run it through the default FP lowering pipeline.

FP lowering can be customized for different subgraphs; the sequence shown here is the recommended workflow for TOSA. Because we are staying in floating-point precision, no calibration with example inputs is required.

If you print the module again, you will see that nodes are left in FP form (or annotated with any necessary casts) without any quantize/dequantize wrappers.

In [ ]:
from executorch.backends.arm.tosa.compile_spec import TosaCompileSpec
from torchao.quantization.pt2e.quantize_pt2e import convert_pt2e, prepare_pt2e
from pathlib import Path

target = "TOSA-1.0+FP"
base_name = "tosa_simple_example"
cwd_dir = Path.cwd()

# Create a compilation spec describing the target for configuring the quantizer
# Dump intermediate artifacts (in this case TOSA flat buffers) to specified location
compile_spec = TosaCompileSpec(target).dump_intermediate_artifacts_to(str(cwd_dir / base_name))

_ = graph_module.print_readable()

# Create a new exported program using the quantized_graph_module
lowered_exported_program = torch.export.export(graph_module, example_inputs)

To lower the graph_module for INT targets using the TOSA backend, we apply the arm_quantizer.

Quantization can be performed in various ways and tailored to different subgraphs; the sequence shown here represents the recommended workflow for TOSA.

This step also requires calibrating the module with representative inputs.

If you print the module again, you’ll see that each node is now wrapped in quantization/dequantization nodes that embed the calculated quantization parameters.

In [ ]:
from executorch.backends.arm.tosa.compile_spec import TosaCompileSpec
from executorch.backends.arm.quantizer import (
    TOSAQuantizer,
    get_symmetric_quantization_config,
)
from torchao.quantization.pt2e.quantize_pt2e import convert_pt2e, prepare_pt2e
from pathlib import Path

target = "TOSA-1.0+INT"
base_name = "tosa_simple_example"
cwd_dir = Path.cwd()

# Create a compilation spec describing the target for configuring the quantizer
# Dump intermediate artifacts (in this case TOSA flat buffers) to specified location
compile_spec = TosaCompileSpec(target).dump_intermediate_artifacts_to(str(cwd_dir / base_name))

# Create and configure quantizer to use a symmetric quantization config globally on all nodes
quantizer = TOSAQuantizer(compile_spec)
operator_config = get_symmetric_quantization_config()
quantizer.set_global(operator_config)

# Post training quantization
quantized_graph_module = prepare_pt2e(graph_module, quantizer)
quantized_graph_module(*example_inputs) # Calibrate the graph module with the example input
quantized_graph_module = convert_pt2e(quantized_graph_module)

_ = quantized_graph_module.print_readable()

# Create a new exported program using the quantized_graph_module
lowered_exported_program = torch.export.export(quantized_graph_module, example_inputs)

The lowering in the TOSABackend happens in four steps:

1. **Lowering to core Aten operator set**: Transform module to use a subset of operators applicable to edge devices. 
2. **Partitioning**: Find subgraphs which are supported for running on TOSA
3. **Lowering to TOSA compatible operator set**: Perform transforms to make the TOSA subgraph(s) compatible with TOSA operator set
4. **Serialization to TOSA**: Compiles the graph module into a TOSA graph 
Step 4 also prints a Network summary for each processed subgraph.

All of this happens behind the scenes in `to_edge_transform_and_lower`. Printing the graph module shows that what is left in the graph is two quantization nodes for `x` and `y` going into an `executorch_call_delegate` node, followed by a dequantization node.

In [ ]:
from executorch.backends.arm.tosa.partitioner import TOSAPartitioner
from executorch.exir import (
    EdgeCompileConfig,
    ExecutorchBackendConfig,
    to_edge_transform_and_lower,
)
from executorch.extension.export_util.utils import save_pte_program

# Create partitioner from compile spec
partitioner = TOSAPartitioner(compile_spec)

# Lower the exported program to the TOSA backend
edge_program_manager = to_edge_transform_and_lower(
            lowered_exported_program,
            partitioner=[partitioner],
            compile_config=EdgeCompileConfig(
                _check_ir_validity=False,
            ),
        )

# Convert edge program to executorch
executorch_program_manager = edge_program_manager.to_executorch(
            config=ExecutorchBackendConfig(extract_delegate_segments=False)
        )

executorch_program_manager.exported_program().module().print_readable()

# Save pte file
pte_name = base_name + ".pte"
pte_path = cwd_dir / base_name / pte_name
save_pte_program(executorch_program_manager, str(pte_path))
assert pte_path.exists(), "Build failed; no .pte-file found"

## Use TOSA reference model to verify TOSA graph

After the AOT compilation flow is done, the resulting lowered TOSA graph can be verified using the TOSA reference model tool.

In [ ]:
import subprocess
import tosa_reference_model as reference_model
from executorch.backends.arm.test.runner_utils import TosaReferenceModelDispatch

# Run TOSA graph through reference model using sample inputs
with TosaReferenceModelDispatch():
    executorch_program_manager.exported_program().module()(*example_inputs)